In [42]:
# import pyodbc
# connect_string = ''
# try:
#     conn = pyodbc.connect('DRIVER={SQL Server};SERVER=10.236.6.111;PORT=1433;DATABASE=Teambuilder;UID=admin;PWD=TeamBuilder;')
#     print("{c} is working".format(c=connect_string))
# except pyodbc.Error as ex:
#     print("{c} is not working".format(c=connect_string))


In [43]:
# import pandas as pd
# ##After connecting to database extract preprocessed information
# #Read in ID and resumes using pandas

# #This is how to create a two column list using two lists newlist = list(zip(numbers,letters))
# #Get data from server
# student_id = []
# student_resume = []
# cursor = conn.cursor()
# cursor.execute("SELECT ALL * FROM StudentInformation") #StudentInformation is the name of the table
# for row in cursor:
#         student_id.append(row.StudentName)
#         student_resume(row.Resume)
# #Store data in pandas data frame
# student_info = list(zip(student_id, student_resume))
# df = pd.DataFrame(student_info, columns=['ID', 'Processed Resume'])
# df.head()

In [44]:
#use the same techniques we used to train to train the model with student names and identites

In [45]:
#Setting Up the Environment
import re #For Preprocessing
import pandas as pd #For Data Handling
from time import time #To Time the Operations
from collections import defaultdict #Use for word frequency

import spacy #More Preprocessing

import logging #Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)
import multiprocessing

from gensim.models import Word2Vec

In [46]:
from tika import parser

df = pd.DataFrame(columns=['CandidateID', 'resume', 'cleaned_resume'])
for dirname , _, filenames in os.walk('test'):
    for filename in filenames:
       path = os.path.join(dirname, filename)
       raw = parser.from_file(path)
       df = df.append({'CandidateID' : filename[:-4], 'resume' : " ".join(raw['content'].strip().split('\n')[1:]), 'cleaned_resume' : ""}, 
                ignore_index = True)
   
df.shape

(4, 3)

In [47]:
def cleanResume(resumeText):
    resumeText = re.sub('http\S+\s*', ' ', resumeText)  # remove URLs
    resumeText = re.sub('RT|cc', ' ', resumeText)  # remove RT and cc
    resumeText = re.sub('#\S+', '', resumeText)  # remove hashtags
    resumeText = re.sub('@\S+', '  ', resumeText)  # remove mentions
    resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resumeText)  # remove punctuations
    resumeText = re.sub(r'[^\x00-\x7f]',r' ', resumeText) 
    resumeText = re.sub('\s+', ' ', resumeText)  # remove extra whitespace
    return resumeText

In [48]:
df['cleaned_resume'] = df.resume.apply(lambda x: cleanResume(x))
df = df.drop('resume', 1)
df.shape

(4, 2)

In [49]:
nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser']) # disabling Named Entity Recognition for speed

def cleaning(doc):
    # Lemmatizes and removes stopwords
    # doc needs to be a spacy Doc object
    txt = [token.lemma_ for token in doc if not token.is_stop]
    # Word2Vec uses context words to learn the vector representation of a target word,
    # if a sentence is only one or two words long,
    # the benefit for the training is very small
    if len(txt) > 2:
        return ' '.join(txt)

In [50]:
brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in df['cleaned_resume'])

In [51]:
t = time()

txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000)]

print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

Time to clean up everything: 0.0 mins


In [52]:
df_clean = pd.DataFrame({'clean': txt})
df_clean = df_clean.dropna().drop_duplicates()
df_clean.shape

(4, 1)

In [53]:
from gensim.models.phrases import Phrases, Phraser

In [54]:
sent = [row.split() for row in df_clean['clean']]

In [55]:
phrases = Phrases(sent, min_count=30, progress_per=10000)

INFO - 19:05:26: collecting all words and their counts
INFO - 19:05:26: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 19:05:26: collected 705 token types (unigram + bigrams) from a corpus of 460 words and 4 sentences
INFO - 19:05:26: merged Phrases<705 vocab, min_count=30, threshold=10.0, max_vocab_size=40000000>
INFO - 19:05:26: Phrases lifecycle event {'msg': 'built Phrases<705 vocab, min_count=30, threshold=10.0, max_vocab_size=40000000> in 0.00s', 'datetime': '2021-10-31T19:05:26.831100', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'created'}


In [56]:
bigram = Phraser(phrases)

INFO - 19:05:26: exporting phrases from Phrases<705 vocab, min_count=30, threshold=10.0, max_vocab_size=40000000>
INFO - 19:05:26: FrozenPhrases lifecycle event {'msg': 'exported FrozenPhrases<0 phrases, min_count=30, threshold=10.0> from Phrases<705 vocab, min_count=30, threshold=10.0, max_vocab_size=40000000> in 0.00s', 'datetime': '2021-10-31T19:05:26.875140', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'created'}


In [57]:
sentences = bigram[sent]

In [58]:
word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

280

In [59]:
sorted(word_freq, key=word_freq.get, reverse=True)[:10]

['datum',
 'work',
 'nlp',
 'learn',
 'machine',
 'python',
 'project',
 'r',
 'e',
 'skill']

In [60]:
import multiprocessing

from gensim.models import Word2Vec

In [61]:
model = Word2Vec.load('basemodel.bin')
model.min_count = 5
model.alpha = 0.03
model.window = 3
print(len(model.wv.vectors))

INFO - 19:05:27: loading Word2Vec object from basemodel.bin
INFO - 19:05:27: loading wv recursively from basemodel.bin.wv.* with mmap=None
INFO - 19:05:27: setting ignored attribute cum_table to None
INFO - 19:05:27: Word2Vec lifecycle event {'fname': 'basemodel.bin', 'datetime': '2021-10-31T19:05:27.115363', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'loaded'}


1273


In [62]:
t = time()

model.build_vocab(sentences, update=True,progress_per=10000,)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))
print(len(model.wv.vectors))

INFO - 19:05:27: collecting all words and their counts
INFO - 19:05:27: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 19:05:27: collected 280 word types from a corpus of 460 raw words and 4 sentences
INFO - 19:05:27: Updating model with new vocabulary
INFO - 19:05:27: Word2Vec lifecycle event {'msg': 'added 1 new unique words (0.35714285714285715%% of original 280) and increased the count of 11 pre-existing words (3.9285714285714284%% of original 280)', 'datetime': '2021-10-31T19:05:27.147395', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'prepare_vocab'}
INFO - 19:05:27: deleting the raw counts dictionary of 280 items
INFO - 19:05:27: sample=6e-05 downsamples 12 most-common words
INFO - 19:05:27: Word2Vec lifecycle event {'msg': 'downsampling leaves estimated 2.763337066447015 word corpus (2.7%% of prior 102)', 'datetime': '2021-10-31T19:05:27.148396', 'ge

Time to build vocab: 0.0 mins
1274


In [63]:
t = time()

model.train(sentences, total_examples=model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))
print(len(model.wv.vectors))

WARNING - 19:05:27: Effective 'alpha' higher than previous training cycles
INFO - 19:05:27: Word2Vec lifecycle event {'msg': 'training model with 23 workers on 1274 vocabulary and 300 features, using sg=0 hs=0 sample=6e-05 negative=20 window=3 shrink_windows=True', 'datetime': '2021-10-31T19:05:27.188433', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'train'}
INFO - 19:05:27: worker thread finished; awaiting finish of 22 more threads
INFO - 19:05:27: worker thread finished; awaiting finish of 21 more threads
INFO - 19:05:27: worker thread finished; awaiting finish of 20 more threads
INFO - 19:05:27: worker thread finished; awaiting finish of 19 more threads
INFO - 19:05:27: worker thread finished; awaiting finish of 18 more threads
INFO - 19:05:27: worker thread finished; awaiting finish of 17 more threads
INFO - 19:05:27: worker thread finished; awaiting finish of 16 more threads
IN

Time to train the model: 0.01 mins
1274


In [71]:
model.wv.most_similar(positive=["machine"])

[('circuit', 0.9861485958099365),
 ('panel', 0.9776694178581238),
 ('oil', 0.9774757623672485),
 ('layout', 0.9744130373001099),
 ('transmission', 0.9727044701576233),
 ('machinery', 0.97134929895401),
 ('diagram', 0.9677846431732178),
 ('simulation', 0.9646748900413513),
 ('dc', 0.9617171287536621),
 ('light', 0.9611549377441406)]